# Data Exploration and Cleaning

This notebook contains some exploration of the dataset and some cleaning for further analysis. The Readme file for the repository contains information on the data source. 

Because the CSV is compressed and present in the repository as a XZ file, and read directly compressed with Pandas.

In [2]:
import pandas as pd

First, we load the data, and explore its structure

In [3]:
df = pd.read_csv("../data/bq-results-20240802-233415-1722641679845.csv.xz", 
                 compression='xz')
df.shape

(138864, 18)

In [4]:
df.head()

,scientific_name,contig_acc,biosample_acc,bioproject_acc,target_acc,element_symbol,protein_acc,type,class,subclass,taxgroup_name,strain,serovar,isolation_source,geo_loc_name,epi_type,host,collection_date
0,Salmonella enterica,AAFUZC010000051.1,SAMN03098832,NaN,PDT000041084.2,blaDHA,NaN,AMR,BETA-LACTAM,CEPHALOSPORIN,Salmonella enterica,AM49198,NaN,NaN,USA,clinical,NaN,NaN
1,Salmonella enterica,AAFUZC010000051.1,SAMN03098832,NaN,PDT000041084.2,ble,EBK1426116.1,AMR,BLEOMYCIN,BLEOMYCIN,Salmonella enterica,AM49198,NaN,NaN,USA,clinical,NaN,NaN
2,Salmonella enterica,AAFUZC010000051.1,SAMN03098832,NaN,PDT000041084.2,blaNDM-1,EBK1426117.1,AMR,BETA-LACTAM,CARBAPENEM,Salmonella enterica,AM49198,NaN,NaN,USA,clinical,NaN,NaN
3,Salmonella enterica,AAFUZC010000073.1,SAMN03098832,NaN,PDT000041084.2,blaNDM-1,EBK1426163.1,AMR,BETA-LACTAM,CARBAPENEM,Salmonella enterica,AM49198,NaN,NaN,USA,clinical,NaN,NaN
4,Salmonella enterica,AAFUZC010000073.1,SAMN03098832,NaN,PDT000041084.2,ble,EBK1426164.1,AMR,BLEOMYCIN,BLEOMYCIN,Salmonella enterica,AM49198,NaN,NaN,USA,clinical,NaN,NaN


In [5]:
df.columns

Index(['scientific_name', 'contig_acc', 'biosample_acc', 'bioproject_acc',
       'target_acc', 'element_symbol', 'protein_acc', 'type', 'class',
       'subclass', 'taxgroup_name', 'strain', 'serovar', 'isolation_source',
       'geo_loc_name', 'epi_type', 'host', 'collection_date'],
      dtype='object')

The dataset has 138,886 rows and 18 columns. The description of the columns is the following:

- scientific_name: the bacterial species name
- contig_acc: the contig accesion number from NCBI, where the gene is present
- biosample_acc: the biosample accession number
- bioproject_acc: the bioproject accession number
- target_acc: unsure, this seems to be an accession number for the genome assembly version
- element_symbol: the gene name (e.g. _blaNDM-1_)
- protein_acc: the protein accession number for the gene
- type: the type of mechanism (e.g. AMR, antimicrobial resistance)
- class: the functional class for the mechanism in this row
- subclass: the functional subclass for the mechanism in this row
- taxgroup_name: The taxonomic group. It may be similar to the scientific name, but unsure at the moment
- strain: the strain for this genome
- serovar: For some taxa, it could be a serovar classification.
- isolation_source: The biological source of the sample (it seems). For example, blood.
- geo_loc_name: the geographical origin of the isolate
- epi_type: needs some exploration, it seems to be the type of strain (clinical, maybe environmental?)
- host: The organism where this bacteria was isolated, for example _Homo sapiens_ (human)
- collection_date: The date where the isolate was collected.

Just from the first five rows, we can see that not all the columns are complete, and we need some additional filtering to have the data ready for analysis.

We can look in more detail the dataset, and use this information for further cleaning, including a description of the dataset, and looking at the NA values present in it

In [8]:
df.describe(include="all")

,scientific_name,contig_acc,biosample_acc,bioproject_acc,target_acc,element_symbol,protein_acc,type,class,subclass,taxgroup_name,strain,serovar,isolation_source,geo_loc_name,epi_type,host,collection_date
count,138864,138864,138864,26693,138864,138864,121804,138864,138007,138007,138864,98238,1983,104426,136244,124544,122637,132082
unique,96,41640,26809,692,26891,525,84585,3,37,64,32,18425,71,1268,907,2,107,3570
top,Escherichia coli,NZ_CP095443.1,SAMN27484493,PRJNA288601,PDT001315396.1,ble,WP_004201167.1,AMR,BETA-LACTAM,CARBAPENEM,E.coli and Shigella,308,not available,urine,USA,clinical,Homo sapiens,2023
freq,58548,109,129,5283,129,40865,7959,123952,46061,41690,58667,776,447,25639,40911,118880,116559,11480


In [9]:
df.isnull().sum()

scientific_name          0
contig_acc               0
biosample_acc            0
bioproject_acc      112171
target_acc               0
element_symbol           0
protein_acc          17060
type                     0
class                  857
subclass               857
taxgroup_name            0
strain               40626
serovar             136881
isolation_source     34438
geo_loc_name          2620
epi_type             14320
host                 16227
collection_date       6782
dtype: int64

One of the first questions is to find a unique identifier that can be used across the dataset. Each entry may have multiple rows associated, and variables such as `scientific_name` can be found multiple times. Looking at the description of the dataset some things that come up to attention:

- biosample_acc: This appears to be the best identifier. From NCBI, this refers is a unique identifier associated with the biological sample. There are no null values in this column.
- bioproject_acc: Not present in all of the rows. Not a good identifier.
- target_acc: according to the [NCBI documentation](https://www.ncbi.nlm.nih.gov/pathogens/pathogens_help/#isolates-browser-data-field-Isolate) it refers to the pathogen detection accession of the isolate. Probably could be used as an ID, but the number is higher compared to the biosample_acc. Maybe there is more than one genome assembly per biosample, this needs to be further explored.
- For further analysis and some of the questions, not all samples have isolation_source, geo_loc_name, host, and date, so we have to keep this in mind.